<a href="https://colab.research.google.com/github/Augustus2001/Datascience/blob/main/Monkeypox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
import zipfile
from google.colab import files
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tf.__version__)

2.8.2


In [2]:
tf.random.set_seed(72)

In [3]:
drive.mount('/content/drive/')
zip_ref = zipfile.ZipFile("/content/drive/My Drive/monkeypox.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
train_MP=os.path.join("/tmp/Fold1/Fold1/Fold1/Train/Monkeypox")
list_train_MP=os.listdir(train_MP)
train_O=os.path.join("/tmp/Fold1/Fold1/Fold1/Train/Others")
list_train_O=os.listdir(train_O)
list_train_MP,list_train_O

(['M47_01_01.jpg',
  'M31_02_06.jpg',
  'M13_01_10.jpg',
  'M31_01_04.jpg',
  'M32_04_05.jpg',
  'M37_01_06.jpg',
  'M28_03_10.jpg',
  'M41_01_09.jpg',
  'M36_01_11.jpg',
  'M29_01_01.jpg',
  'M35_01_03.jpg',
  'M54_01_05.jpg',
  'M09_04_03.jpg',
  'M33_01_00.jpg',
  'M32_04_09.jpg',
  'M32_01_05.jpg',
  'M20_01_04.jpg',
  'M18_01_07.jpg',
  'M21_02_02.jpg',
  'M22_01_09.jpg',
  'M52_01_02.jpg',
  'M29_01_09.jpg',
  'M31_02_08.jpg',
  'M47_01_06.jpg',
  'M01_04_00.jpg',
  'M42_02_02.jpg',
  'M11_01_03.jpg',
  'M44_01_04.jpg',
  'M49_01_05.jpg',
  'M47_01_09.jpg',
  'M05_01_03.jpg',
  'M49_01_04.jpg',
  'M33_01_06.jpg',
  'M46_02_09.jpg',
  'M32_01_13.jpg',
  'M12_01_13.jpg',
  'M05_02_00.jpg',
  'M01_04_02.jpg',
  'M20_02_12.jpg',
  'M16_01_13.jpg',
  'M09_01_11.jpg',
  'M33_01_02.jpg',
  'M21_02_06.jpg',
  'M01_01_07.jpg',
  'M31_02_02.jpg',
  'M06_02_00.jpg',
  'M24_01_09.jpg',
  'M32_02_00.jpg',
  'M20_01_12.jpg',
  'M46_02_05.jpg',
  'M52_01_13.jpg',
  'M32_01_12.jpg',
  'M28_04_03

In [5]:
train_datagen=ImageDataGenerator(rescale=1/255.,
                                    rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=False,
                                    )

In [6]:
train_generator=train_datagen.flow_from_directory(directory="/tmp/Fold1/Fold1/Fold1/Train",
                                                   target_size=(244,244),
                                                   class_mode="binary",
                                                  batch_size=32)
test_datagen=ImageDataGenerator()

Found 2142 images belonging to 2 classes.


In [7]:
test_generator=test_datagen.flow_from_directory(directory="/tmp/Fold1/Fold1/Fold1/Test",
                                                   target_size=(244,244),
                                                   class_mode="binary",
                                            )

Found 45 images belonging to 2 classes.


In [8]:
from keras.backend import dropout
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),activation="relu",input_shape=(244,244,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv2D(64,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 242, 242, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 121, 121, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 121, 121, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 119, 119, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 59, 59, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 59, 59, 64)        0

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.7),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator,epochs=40,steps_per_epoch=20)

Epoch 1/40
12/20 [=================>............] - ETA: 44s - loss: 28.8571 - accuracy: 0.5469

In [ ]:
from tensorflow.keras import backend as K
K.clear_session()